In [1]:
%load_ext autoreload
%autoreload 2

Try classical SVM

In [2]:
#scikit learn and tools
import pandas as pd
import numpy as np;
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from embedding import kernel_matrix

In [3]:
#load dataset with panda
#data are scaled outside the notebook
env = pd.read_csv('env.sel2.scaled.csv')  

In [4]:
#take a look of the dataset
env.describe()

,illuminance,blinds,lamps,co,rh,co2,temp,occupancy
count,1.093400e+04,10934.000000,10934.000000,1.093400e+04,10934.000000,1.093400e+04,1.093400e+04,10934.000000
mean,2.859327e-17,0.803111,0.504677,-1.871559e-16,0.471725,1.455657e-16,-4.159021e-16,-0.315164
std,1.000000e+00,0.371107,0.498983,1.000000e+00,0.076689,1.000000e+00,1.000000e+00,0.949081
min,-9.310658e-01,0.000000,0.000000,-8.039283e-01,0.243300,-1.145063e+00,-1.711094e+00,-1.000000
25%,-8.848906e-01,1.000000,0.000000,-7.808253e-01,0.431300,-7.923978e-01,-8.078199e-01,-1.000000
50%,2.542024e-02,1.000000,1.000000,-2.725584e-01,0.487400,-3.964529e-01,-1.153100e-01,-1.000000
75%,3.882253e-01,1.000000,1.000000,3.743267e-01,0.519800,6.966369e-01,7.428010e-01,1.000000
max,9.814560e+00,1.000000,1.000000,6.542838e+00,0.633300,3.950372e+00,2.594513e+00,1.000000


In [5]:
#DEFINE design matrix
Y = env['occupancy']
#X = env[['illuminance', 'blinds','lamps','co', 'rh', 'co2', 'temp']]
X = env[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]

X.head()

,illuminance,blinds,lamps,rh,co2,temp
0,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
1,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
2,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
3,-0.90468,1.0,0.0,0.4069,-0.795139,-0.030001
4,-0.90468,1.0,0.0,0.4069,-0.795139,-0.030001


In [6]:
#split design matrix (25% of the design matrix used for test)
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
#sanity check
print('TRAIN*****')
display(X_train.head())
display(X_train.describe())
print('TEST*****')
display(X_test.head())
display(X_test.describe())

TRAIN*****


,illuminance,blinds,lamps,rh,co2,temp
1903,0.401418,0.68,1.0,0.2598,-0.770075,2.594513
3129,2.914668,1.00,1.0,0.5094,2.021188,-0.245782
9625,4.590167,1.00,1.0,0.5106,1.345924,-0.546874
6630,-0.858505,1.00,0.0,0.5350,-0.403743,-1.364839
6194,0.157349,1.00,1.0,0.5241,1.552783,-0.762656


,illuminance,blinds,lamps,rh,co2,temp
count,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000
mean,0.002035,0.802176,0.505624,0.472100,0.004443,0.000076
std,1.009702,0.371952,0.498979,0.076677,1.002684,1.000814
min,-0.931066,0.000000,0.000000,0.243300,-1.145063,-1.711094
25%,-0.884891,1.000000,0.000000,0.433700,-0.792398,-0.807820
50%,0.032017,1.000000,1.000000,0.487400,-0.395539,-0.115310
75%,0.388225,1.000000,1.000000,0.519800,0.698455,0.742801
max,9.814560,1.000000,1.000000,0.633300,3.950372,2.594513


TEST*****


,illuminance,blinds,lamps,rh,co2,temp
1988,0.071595,1.0,1.0,0.2927,-0.350884,1.691239
2133,-0.931066,1.0,0.0,0.3678,-0.766430,1.219529
6249,-0.152684,1.0,0.0,0.5338,1.206956,-0.717492
10389,0.064999,1.0,1.0,0.5326,0.487950,-0.807820
7156,-0.931066,1.0,0.0,0.4581,-0.772354,-0.331091


,illuminance,blinds,lamps,rh,co2,temp
count,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000
mean,-0.006104,0.805914,0.501836,0.470599,-0.013325,-0.000228
std,0.970475,0.368614,0.499076,0.076728,0.991969,0.997738
min,-0.931066,0.000000,0.000000,0.243300,-1.145063,-1.711094
25%,-0.878294,1.000000,0.000000,0.424150,-0.791946,-0.807820
50%,0.018824,1.000000,0.750000,0.486800,-0.399410,-0.115310
75%,0.368436,1.000000,1.000000,0.519800,0.669293,0.742801
max,6.344824,1.000000,1.000000,0.633300,3.672889,2.594513


In [8]:
#WARNING: convert data to numpy. Quantum stuff (Qiskit) do not like PANDAS
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()


In [9]:
#check the shape of test and training dataset
print(f'Training shape dataset {X_train_np.shape}')
print(f'Label for traing {y_train_np.shape}')

print(f'Test shape dataset {X_test_np.shape}')
print(f'Label for test {y_test_np.shape}')

Training shape dataset (8200, 6)
Label for traing (8200,)
Test shape dataset (2734, 6)
Label for test (2734,)


In [10]:
#define data for training
K = 1000
K_test = 1000
#K = len(y_train_np)
#K_test = len(y_test_np)

print(f'Take {K} sample for training and {K_test} for test')


Take 500 sample for training and 100 for test


In [11]:
#try SVM using RBF kernel
svm = SVC(kernel='rbf').fit(X_train_np[:K], y_train_np[:K]);
#result...
predictions = svm.predict(X_test_np[:K_test])
accuracy_score(predictions, y_test[:K_test])

0.81

Try quantum Kernel Method

In [12]:
#Create a cotainer
from embedding import CircuitContainer
from embedding import QEmbedding

c = CircuitContainer(f_embedding= QEmbedding.createQuantumEmbedding, nwire=6, obs=['ZIIIII', 'ZIIZII','ZZZZZZ', 'IZIIII','ZZIIII','ZZIIZI'])

*** Create a Container ***
*** Created quantum template for feature map using 6 qubit ***
     ┌───┐┌───────────┐                         ┌───┐ ░ ┌───┐
q_0: ┤ H ├┤ Rz(phi_0) ├──■──────────────────────┤ X ├─░─┤ H ├
     ├───┤├───────────┤┌─┴─┐                    └─┬─┘ ░ ├───┤
q_1: ┤ H ├┤ Rz(phi_1) ├┤ X ├──■───────────────────┼───░─┤ H ├
     ├───┤├───────────┤└───┘┌─┴─┐                 │   ░ ├───┤
q_2: ┤ H ├┤ Rz(phi_2) ├─────┤ X ├──■──────────────┼───░─┤ H ├
     ├───┤├───────────┤     └───┘┌─┴─┐            │   ░ ├───┤
q_3: ┤ H ├┤ Rz(phi_3) ├──────────┤ X ├──■─────────┼───░─┤ H ├
     ├───┤├───────────┤          └───┘┌─┴─┐       │   ░ ├───┤
q_4: ┤ H ├┤ Rz(phi_4) ├───────────────┤ X ├──■────┼───░─┤ H ├
     ├───┤├───────────┤               └───┘┌─┴─┐  │   ░ ├───┤
q_5: ┤ H ├┤ Rz(phi_5) ├────────────────────┤ X ├──■───░─┤ H ├
     └───┘└───────────┘                    └───┘      ░ └───┘
*** Required observables: ['ZIIIII', 'ZIIZII', 'ZZZZZZ', 'IZIIII', 'ZZIIII', 'ZZIIZI']


In [13]:
#take info about the current container
c.metadata()

*** Quantum template for feature map using 6 qubit ***
     ┌───┐┌───────────┐                         ┌───┐ ░ ┌───┐
q_0: ┤ H ├┤ Rz(phi_0) ├──■──────────────────────┤ X ├─░─┤ H ├
     ├───┤├───────────┤┌─┴─┐                    └─┬─┘ ░ ├───┤
q_1: ┤ H ├┤ Rz(phi_1) ├┤ X ├──■───────────────────┼───░─┤ H ├
     ├───┤├───────────┤└───┘┌─┴─┐                 │   ░ ├───┤
q_2: ┤ H ├┤ Rz(phi_2) ├─────┤ X ├──■──────────────┼───░─┤ H ├
     ├───┤├───────────┤     └───┘┌─┴─┐            │   ░ ├───┤
q_3: ┤ H ├┤ Rz(phi_3) ├──────────┤ X ├──■─────────┼───░─┤ H ├
     ├───┤├───────────┤          └───┘┌─┴─┐       │   ░ ├───┤
q_4: ┤ H ├┤ Rz(phi_4) ├───────────────┤ X ├──■────┼───░─┤ H ├
     ├───┤├───────────┤               └───┘┌─┴─┐  │   ░ ├───┤
q_5: ┤ H ├┤ Rz(phi_5) ├────────────────────┤ X ├──■───░─┤ H ├
     └───┘└───────────┘                    └───┘      ░ └───┘
*** Required observables: ['ZIIIII', 'ZIIZII', 'ZZZZZZ', 'IZIIII', 'ZZIIII', 'ZZIIZI']


In [14]:
#IF NEEDED, REBUILD THE CONTAINER
#c.build(f_embedding= QEmbedding.createQuantumEmbedding, nwire= 6, obs=['ZIIIII', 'ZIIZII','ZZZZZZ', 'IZIIII','ZZIIII','ZZIIZI'])

#c.build(f_embedding= QEmbedding.createQuantumEmbedding, nwire= 6, obs=['ZIIIII'])

In [15]:
#try SVM using linear kernel (dot product)
svm_quantum = SVC(kernel=kernel_matrix).fit(X_train_np[:K], y_train_np[:K]);

In [16]:
q_predictions = svm_quantum.predict(X_test_np[:K_test])
accuracy_score(q_predictions, y_test_np[:K_test])

0.76

In [17]:
#try embedding on single sample
from embedding import qfKernel

x1 = X_train_np[0]
x2 = X_train_np[1]
quantum_kernel = qfKernel(x1, x2)
classic_dot_product = x1.dot(x2)

print(f'Quantum kernel: {quantum_kernel}')
print(f'Classic dot product: {classic_dot_product}')

#check if different from classic
if quantum_kernel == classic_dot_product:
    print('Is the same that clasical')
else:
    print('Is not the same taht classical')

simmetric_kernel = qfKernel(x2,x1)
print(sim_kernel)

if simmetric_kernel == quantum_kernel:
    print('Is ismmetric') 





Quantum kernel: 1.2675948470050888
Classic dot product: 0.788190898707897
Is not the same taht classical


NameError: name 'sim_kernel' is not defined